In [1]:
import os
import glob
import numpy as np
import xarray as xr

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.gridspec as gridspec
%matplotlib inline
import matplotlib
matplotlib.rcParams.update({'font.size':15})
import time
import cartopy.crs as ccrs
import cartopy
CMIP6_FOLDER = "/global/cfs/projectdirs/m3522/cmip6/"
SCRATCH="/pscratch/sd/q/qnicolas/"

import sys
p = os.path.abspath('/global/homes/q/qnicolas/')
if p not in sys.path:
    sys.path.append(p)

from tools.generalTools import *; from tools.BLtools import *; from tools.e5tools import *
from tropicalOrographicRegions.mountainUtils import tilted_rect

from scipy.stats import linregress
#xr.set_options(display_style='text') 

from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from matplotlib.legend_handler import HandlerLine2D
import matplotlib.lines
from scipy.ndimage import gaussian_filter

%load_ext autoreload
%autoreload 2

/global/common/software/m3310/conda/xrdask.2023.11.qn/lib/python3.9/site-packages/pyproj/__init__.py:89: UserWarning: pyproj unable to set database path.
  _pyproj_global_context_initialize()


## With google cloud

In [2]:
import fsspec

In [30]:
df = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')
df.head()

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
0,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,ps,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
1,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,rsds,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
2,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,rlus,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
3,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,rlds,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
4,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,psl,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706


In [31]:
df_uas_hist = df.query("activity_id=='CMIP' & table_id == 'Amon' & variable_id == 'uas' & experiment_id == 'historical'")
df_uas_ssp5 = df.query("activity_id=='ScenarioMIP' & table_id == 'Amon' & variable_id == 'uas' & experiment_id == 'ssp585'")

In [32]:
def subselect(first_table,second_table):
    source_ids = second_table['source_id'].unique()
    mask = first_table['source_id'].isin(source_ids)
    return first_table[mask]

df_uas_hist_itsct = subselect(df_uas_hist,df_uas_ssp5)
df_uas_ssp5_itsct = subselect(df_uas_ssp5,df_uas_hist)

In [33]:
df_vas_hist = df.query("activity_id=='CMIP' & table_id == 'Amon' & variable_id == 'vas' & experiment_id == 'historical'")
df_vas_ssp5 = df.query("activity_id=='ScenarioMIP' & table_id == 'Amon' & variable_id == 'vas' & experiment_id == 'ssp585'")
df_vas_hist_itsct = subselect(df_vas_hist,df_vas_ssp5)
df_vas_ssp5_itsct = subselect(df_vas_ssp5,df_vas_hist)

In [ ]:
df_uas_hist_itsct

In [37]:
df_vas_hist_itsct.shape

(477, 11)

In [38]:
df_uas_hist_itsct.shape

(474, 11)

In [39]:
df_vas_hist_itsct

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
9915,CMIP,NOAA-GFDL,GFDL-CM4,historical,r1i1p1f1,Amon,vas,gr1,gs://cmip6/CMIP6/CMIP/NOAA-GFDL/GFDL-CM4/histo...,NaN,20180701
22186,CMIP,IPSL,IPSL-CM6A-LR,historical,r2i1p1f1,Amon,vas,gr,gs://cmip6/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/histor...,NaN,20180803
22372,CMIP,IPSL,IPSL-CM6A-LR,historical,r30i1p1f1,Amon,vas,gr,gs://cmip6/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/histor...,NaN,20180803
22542,CMIP,IPSL,IPSL-CM6A-LR,historical,r8i1p1f1,Amon,vas,gr,gs://cmip6/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/histor...,NaN,20180803
22855,CMIP,IPSL,IPSL-CM6A-LR,historical,r29i1p1f1,Amon,vas,gr,gs://cmip6/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/histor...,NaN,20180803
...,...,...,...,...,...,...,...,...,...,...,...
522955,CMIP,MRI,MRI-ESM2-0,historical,r7i1p1f1,Amon,vas,gn,gs://cmip6/CMIP6/CMIP/MRI/MRI-ESM2-0/historica...,NaN,20210813
523301,CMIP,MRI,MRI-ESM2-0,historical,r6i1p1f1,Amon,vas,gn,gs://cmip6/CMIP6/CMIP/MRI/MRI-ESM2-0/historica...,NaN,20210907
523709,CMIP,CMCC,CMCC-CM2-SR5,historical,r3i1p2f1,Amon,vas,gn,gs://cmip6/CMIP6/CMIP/CMCC/CMCC-CM2-SR5/histor...,NaN,20211108
523734,CMIP,CMCC,CMCC-CM2-SR5,historical,r2i1p2f1,Amon,vas,gn,gs://cmip6/CMIP6/CMIP/CMCC/CMCC-CM2-SR5/histor...,NaN,20211109


In [20]:
df = df_uas_ssp5_itsct
source_id_map = {source_id: i for i, source_id in enumerate(df['source_id'].unique())}

# group the DataFrame by source_id and count the number of rows in each group
counts = df.groupby('source_id').size().reset_index()
counts.columns = ['source_id', 'count']

# sort the DataFrame by source_id using the original order
counts = counts.sort_values('source_id', key=lambda x: x.map(source_id_map), kind='mergesort')
counts

,source_id,count
18,GFDL-CM4,1
19,GFDL-ESM4,1
8,CNRM-CM6-1,6
3,BCC-CSM2-MR,1
10,CNRM-ESM2-1,5
11,CanESM5,50
12,CanESM5-CanOE,3
37,UKESM1-0-LL,8
2,AWI-CM-1-1-MR,1
25,INM-CM4-8,1


In [34]:
df = df_vas_ssp5_itsct
source_id_map = {source_id: i for i, source_id in enumerate(df['source_id'].unique())}

# group the DataFrame by source_id and count the number of rows in each group
counts = df.groupby('source_id').size().reset_index()
counts.columns = ['source_id', 'count']

# sort the DataFrame by source_id using the original order
counts = counts.sort_values('source_id', key=lambda x: x.map(source_id_map), kind='mergesort')
counts

,source_id,count
18,GFDL-CM4,1
19,GFDL-ESM4,1
8,CNRM-CM6-1,6
3,BCC-CSM2-MR,1
10,CNRM-ESM2-1,5
11,CanESM5,50
12,CanESM5-CanOE,3
37,UKESM1-0-LL,8
2,AWI-CM-1-1-MR,1
25,INM-CM4-8,1


In [19]:
df = df_uas_hist_itsct
source_id_map = {source_id: i for i, source_id in enumerate(df['source_id'].unique())}

# group the DataFrame by source_id and count the number of rows in each group
counts = df.groupby('source_id').size().reset_index()
counts.columns = ['source_id', 'count']

# sort the DataFrame by source_id using the original order
counts = counts.sort_values('source_id', key=lambda x: x.map(source_id_map), kind='mergesort')
counts

,source_id,count
18,GFDL-CM4,1
27,IPSL-CM6A-LR,32
20,GISS-E2-1-G,46
8,CNRM-CM6-1,30
3,BCC-CSM2-MR,3
10,CNRM-ESM2-1,10
32,MIROC6,50
35,MRI-ESM2-0,11
21,GISS-E2-1-H,25
37,UKESM1-0-LL,19


In [18]:
df_uas_hist_itsct.groupby('source_id').size()

source_id
ACCESS-CM2           5
ACCESS-ESM1-5       36
AWI-CM-1-1-MR        5
BCC-CSM2-MR          3
CAMS-CSM1-0          3
CAS-ESM2-0           4
CMCC-CM2-SR5         3
CMCC-ESM2            1
CNRM-CM6-1          30
CNRM-CM6-1-HR        1
CNRM-ESM2-1         10
CanESM5             65
CanESM5-CanOE        3
EC-Earth3           22
EC-Earth3-CC         1
EC-Earth3-Veg        8
EC-Earth3-Veg-LR     3
FGOALS-f3-L          3
GFDL-CM4             1
GFDL-ESM4            1
GISS-E2-1-G         46
GISS-E2-1-H         25
HadGEM3-GC31-LL      5
HadGEM3-GC31-MM      4
IITM-ESM             1
INM-CM4-8            1
INM-CM5-0           10
IPSL-CM6A-LR        32
KACE-1-0-G           3
KIOST-ESM            1
MCM-UA-1-0           2
MIROC-ES2L          31
MIROC6              50
MPI-ESM1-2-HR       10
MPI-ESM1-2-LR       10
MRI-ESM2-0          11
NESM3                5
UKESM1-0-LL         19
dtype: int64

In [21]:
test1 = ["CMIP.historical.AWI.AWI-CM-1-1-MR.Amon.uas.1980-1999.zarr","CMIP.historical.BCC.BCC-CSM2-MR.Amon.uas","CMIP.historical.CAMS.CAMS-CSM1-0.Amon.uas","CMIP.historical.CAS.CAS-ESM2-0.Amon.uas.1980-1999.zarr","CMIP.historical.CAS.FGOALS-f3-L.Amon.uas.1980-1999.zarr","CMIP.historical.CCCma.CanESM5.Amon.uas","CMIP.historical.CCCma.CanESM5-CanOE.Amon.uas","CMIP.historical.CCCR-IITM.IITM-ESM.Amon.uas.1980-1999.zarr","CMIP.historical.CMCC.CMCC-CM2-SR5.Amon.uas.1980-1999.zarr","CMIP.historical.CMCC.CMCC-ESM2.Amon.uas.1980-1999.zarr","CMIP.historical.CNRM-CERFACS.CNRM-CM6-1.Amon.uas","CMIP.historical.CNRM-CERFACS.CNRM-CM6-1-HR.Amon.uas.1980-1999.zarr","CMIP.historical.CNRM-CERFACS.CNRM-ESM2-1.Amon.uas","CMIP.historical.CSIRO.ACCESS-ESM1-5.Amon.uas.1980-1999.zarr","CMIP.historical.CSIRO-ARCCSS.ACCESS-CM2.Amon.uas.1980-1999.zarr","CMIP.historical.EC-Earth-Consortium.EC-Earth3.Amon.uas.1980-1999.zarr","CMIP.historical.EC-Earth-Consortium.EC-Earth3-CC.Amon.uas.1980-1999.zarr","CMIP.historical.EC-Earth-Consortium.EC-Earth3-Veg.Amon.uas.1980-1999.zarr","CMIP.historical.EC-Earth-Consortium.EC-Earth3-Veg-LR.Amon.uas.1980-1999.zarr","CMIP.historical.INM.INM-CM4-8.Amon.uas","CMIP.historical.INM.INM-CM5-0.Amon.uas","CMIP.historical.IPSL.IPSL-CM6A-LR.Amon.uas","CMIP.historical.KIOST.KIOST-ESM.Amon.uas.1980-1999.zarr","CMIP.historical.MIROC.MIROC6.Amon.uas","CMIP.historical.MIROC.MIROC-ES2L.Amon.uas.1980-1999.zarr","CMIP.historical.MOHC.HadGEM3-GC31-LL.Amon.uas","CMIP.historical.MOHC.HadGEM3-GC31-MM.Amon.uas.1980-1999.zarr","CMIP.historical.MOHC.UKESM1-0-LL.Amon.uas","CMIP.historical.MPI-M.MPI-ESM1-2-HR.Amon.uas","CMIP.historical.MPI-M.MPI-ESM1-2-LR.Amon.uas","CMIP.historical.MRI.MRI-ESM2-0.Amon.uas","CMIP.historical.NASA-GISS.GISS-E2-1-G.Amon.uas","CMIP.historical.NASA-GISS.GISS-E2-1-H.Amon.uas","CMIP.historical.NIMS-KMA.KACE-1-0-G.Amon.uas.1980-1999.zarr","CMIP.historical.NOAA-GFDL.GFDL-CM4.Amon.uas","CMIP.historical.NOAA-GFDL.GFDL-ESM4.Amon.uas","CMIP.historical.NUIST.NESM3.Amon.uas","CMIP.historical.UA.MCM-UA-1-0.Amon.uas"]
test2 = ["ScenarioMIP.ssp585.AWI.AWI-CM-1-1-MR.Amon.uas.2080-2099.zarr","ScenarioMIP.ssp585.BCC.BCC-CSM2-MR.Amon.uas.2080-2099.zarr","ScenarioMIP.ssp585.CAMS.CAMS-CSM1-0.Amon.uas.2080-2099.zarr","ScenarioMIP.ssp585.CAS.CAS-ESM2-0.Amon.uas.2080-2099.zarr","ScenarioMIP.ssp585.CAS.FGOALS-f3-L.Amon.uas.2080-2099.zarr","ScenarioMIP.ssp585.CCCma.CanESM5.Amon.uas.2080-2099.zarr","ScenarioMIP.ssp585.CCCma.CanESM5-CanOE.Amon.uas.2080-2099.zarr","ScenarioMIP.ssp585.CCCR-IITM.IITM-ESM.Amon.uas.2080-2099.zarr","ScenarioMIP.ssp585.CMCC.CMCC-CM2-SR5.Amon.uas.2080-2099.zarr","ScenarioMIP.ssp585.CMCC.CMCC-ESM2.Amon.uas.2080-2099.zarr","ScenarioMIP.ssp585.CNRM-CERFACS.CNRM-CM6-1.Amon.uas.2080-2099.zarr","ScenarioMIP.ssp585.CNRM-CERFACS.CNRM-CM6-1-HR.Amon.uas.2080-2099.zarr","ScenarioMIP.ssp585.CNRM-CERFACS.CNRM-ESM2-1.Amon.uas.2080-2099.zarr","ScenarioMIP.ssp585.CSIRO.ACCESS-ESM1-5.Amon.uas.2080-2099.zarr","ScenarioMIP.ssp585.CSIRO-ARCCSS.ACCESS-CM2.Amon.uas.2080-2099.zarr","ScenarioMIP.ssp585.DKRZ.MPI-ESM1-2-HR.Amon.uas.2080-2099.zarr","ScenarioMIP.ssp585.EC-Earth-Consortium.EC-Earth3.Amon.uas.2080-2099.zarr","ScenarioMIP.ssp585.EC-Earth-Consortium.EC-Earth3-CC.Amon.uas.2080-2099.zarr","ScenarioMIP.ssp585.EC-Earth-Consortium.EC-Earth3-Veg.Amon.uas.2080-2099.zarr","ScenarioMIP.ssp585.EC-Earth-Consortium.EC-Earth3-Veg-LR.Amon.uas.2080-2099.zarr","ScenarioMIP.ssp585.INM.INM-CM4-8.Amon.uas.2080-2099.zarr","ScenarioMIP.ssp585.INM.INM-CM5-0.Amon.uas.2080-2099.zarr","ScenarioMIP.ssp585.IPSL.IPSL-CM6A-LR.Amon.uas.2080-2099.zarr","ScenarioMIP.ssp585.KIOST.KIOST-ESM.Amon.uas.2080-2099.zarr","ScenarioMIP.ssp585.MIROC.MIROC6.Amon.uas.2080-2099.zarr","ScenarioMIP.ssp585.MIROC.MIROC-ES2L.Amon.uas.2080-2099.zarr","ScenarioMIP.ssp585.MOHC.HadGEM3-GC31-LL.Amon.uas.2080-2099.zarr","ScenarioMIP.ssp585.MOHC.HadGEM3-GC31-MM.Amon.uas.2080-2099.zarr","ScenarioMIP.ssp585.MOHC.UKESM1-0-LL.Amon.uas.2080-2099.zarr","ScenarioMIP.ssp585.MPI-M.MPI-ESM1-2-LR.Amon.uas.2080-2099.zarr","ScenarioMIP.ssp585.MRI.MRI-ESM2-0.Amon.uas.2080-2099.zarr","ScenarioMIP.ssp585.NASA-GISS.GISS-E2-1-G.Amon.uas.2080-2099.zarr","ScenarioMIP.ssp585.NASA-GISS.GISS-E2-1-H.Amon.uas.2080-2099.zarr","ScenarioMIP.ssp585.NIMS-KMA.KACE-1-0-G.Amon.uas.2080-2099.zarr","ScenarioMIP.ssp585.NOAA-GFDL.GFDL-CM4.Amon.uas.2080-2099.zarr","ScenarioMIP.ssp585.NOAA-GFDL.GFDL-ESM4.Amon.uas.2080-2099.zarr","ScenarioMIP.ssp585.NUIST.NESM3.Amon.uas.2080-2099.zarr","ScenarioMIP.ssp585.UA.MCM-UA-1-0.Amon.uas.2080-2099.zarr"]

In [27]:
for t in test1:
    if t[-4:]!='zarr':
        print('mv',t,t+'1980-1999.zarr')

mv CMIP.historical.BCC.BCC-CSM2-MR.Amon.uas CMIP.historical.BCC.BCC-CSM2-MR.Amon.uas1980-1999.zarr
mv CMIP.historical.CAMS.CAMS-CSM1-0.Amon.uas CMIP.historical.CAMS.CAMS-CSM1-0.Amon.uas1980-1999.zarr
mv CMIP.historical.CCCma.CanESM5.Amon.uas CMIP.historical.CCCma.CanESM5.Amon.uas1980-1999.zarr
mv CMIP.historical.CCCma.CanESM5-CanOE.Amon.uas CMIP.historical.CCCma.CanESM5-CanOE.Amon.uas1980-1999.zarr
mv CMIP.historical.CNRM-CERFACS.CNRM-CM6-1.Amon.uas CMIP.historical.CNRM-CERFACS.CNRM-CM6-1.Amon.uas1980-1999.zarr
mv CMIP.historical.CNRM-CERFACS.CNRM-ESM2-1.Amon.uas CMIP.historical.CNRM-CERFACS.CNRM-ESM2-1.Amon.uas1980-1999.zarr
mv CMIP.historical.INM.INM-CM4-8.Amon.uas CMIP.historical.INM.INM-CM4-8.Amon.uas1980-1999.zarr
mv CMIP.historical.INM.INM-CM5-0.Amon.uas CMIP.historical.INM.INM-CM5-0.Amon.uas1980-1999.zarr
mv CMIP.historical.IPSL.IPSL-CM6A-LR.Amon.uas CMIP.historical.IPSL.IPSL-CM6A-LR.Amon.uas1980-1999.zarr
mv CMIP.historical.MIROC.MIROC6.Amon.uas CMIP.historical.MIROC.MIROC6.Amo